In [ ]:
!pip install aiohttp nest_asyncio
!pip install pyngrok
!pip install python-dotenv
!pip install psutil
!pip install gitpython

In [ ]:
import psutil
import os

def kill_port(port):
    for proc in psutil.process_iter(['pid', 'name']):
        for conn in proc.connections(kind='inet'):
            if conn.laddr.port == port:
                os.kill(proc.info['pid'], 9)  # Beende den Prozess auf dem angegebenen Port
                print(f"Prozess {proc.info['pid']} auf Port {port} wurde beendet.")

# Port 8081 freigeben, falls er belegt ist
kill_port(8081)

In [ ]:
#EASY MODE

!pip install flask pyngrok python-dotenv

import os
from flask import Flask, jsonify
from pyngrok import ngrok
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus .env-Datei
load_dotenv()

app = Flask(__name__)

# Health-Check Route
@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy'})

if __name__ == "__main__":
    # Hole die ngrok Authtoken und den Port aus den Umgebungsvariablen
    authtoken = os.getenv("NGROK_AUTHTOKEN")
    print(f"Der Authtoken ist: {authtoken}")
    port = int(os.getenv("PORT", 5000))
    print(f"Der Port ist: {port}")

    # Starte ngrok und öffne den Tunnel
    ngrok.set_auth_token(authtoken)
    public_url = ngrok.connect(port)
    print(f"ngrok-Tunnel geöffnet unter {public_url}")

    # Starte den Flask-Server auf dem angegebenen Port
    app.run(port=port)

In [ ]:
#START mit ngrok
# start_server.py
import asyncio
from prozessor_utils.flask_server import app
from prozessor_utils.ngrok_utils import start_ngrok
from dotenv import load_dotenv
import os

load_dotenv()

async def start_services():
    try:
        authtoken = os.getenv("NGROK_AUTHTOKEN")
        port = int(os.getenv("PORT", 5000))
        await start_ngrok(authtoken, port)
    except Exception as e:
        print(f"Fehler beim Starten der Dienste: {str(e)}")

if __name__ == "__main__":
    port = int(os.getenv("PORT", 5000))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(start_services())
    app.run(port=port)
